In [1]:
#!pip install ibm_ai_openscale

In [1]:
from ibm_ai_openscale import APIClient
from ibm_ai_openscale.supporting_classes import PayloadRecord

# Guide to create AIOS credentials -> https://cloud.ibm.com/docs/services/ai-openscale?topic=ai-openscale-cred-create
aios_credentials = {
                      "instance_guid": "<your openscale instance id>",
                      "apikey": "***",
                      "url": "https://api.aiopenscale.cloud.ibm.com"
                   }

client = APIClient(aios_credentials)

In [2]:
client.data_mart.subscriptions.list()

9f4c4392-3eb8-4717-8104-2918b941ac83,[asset] Custom WA Wrapper,model,cba14800-7f2a-11ea-a163-1788a632d4f7,2020-04-15T15:08:49.855Z
26ea7058-e84a-4bf0-91cc-865aa294b542,Go Sales - 06 Apr 2020 18:40:12,model,9ce13ef2-2c7f-466a-b73a-f80322a3b317,2020-04-06T13:11:14.192Z
000e052e-7323-40a7-8307-d2b18689a16c,Go Sales - 02 Apr 2020 18:42:43,model,9ce13ef2-2c7f-466a-b73a-f80322a3b317,2020-04-02T13:13:48.500Z


In [3]:
subscription_id = "9f4c4392-3eb8-4717-8104-2918b941ac83"
subscription = client.data_mart.subscriptions.get(subscription_uid=subscription_id)

In [4]:
%%time
import requests

url = "<your custom deployment scoring endpoint>"

payload = {
    "fields": ["Reviews"],
    "values": [
        ["Great phone available at an affordable price. Amazing performance and battery life make it an excellent choice."],
        ["Very disappointed. Screen barely responds to touch, Battery is taking forever to charge and dies very fast. Useless device."]
    ]
}

response = requests.post(url, json=payload, auth=('admin', 'password'))

CPU times: user 19 ms, sys: 4.67 ms, total: 23.7 ms
Wall time: 2.51 s


In [5]:
response.status_code, response.text

(200,
 '{"fields":["Reviews","top_intent","confidence"],"values":[["Great phone available at an affordable price. Amazing performance and battery life make it an excellent choice.","Very_Good",[0.1284218349229962,0.11798729970815604,0.11146429820166036,0.10733957805963275,0.5347869891075547]],["Very disappointed. Screen barely responds to touch, Battery is taking forever to charge and dies very fast. Useless device.","Very_Bad",[0.12313512969153419,0.11768187924517681,0.12954178725467652,0.519235472319682,0.11040573148893042]]]}')

In [6]:
records = [PayloadRecord(request=payload, response=response.json())]

subscription.payload_logging.store(records=records)

In [7]:
import time

time.sleep(5)
payload_table = subscription.payload_logging.get_table_content()

In [8]:
payload_table

,scoring_id,scoring_timestamp,deployment_id,asset_revision,Reviews,top_intent,confidence,prediction_probability
0,e2cdb593-4731-4c62-af40-389d7709f1eb-1,2020-04-15 15:12:45.202000+00:00,21deb68d-084a-4389-be5c-6cd13b9e3e9c,None,Great phone available at an affordable price. ...,Very_Good,"[0.1284218349229962, 0.11798729970815604, 0.11...",None
1,e2cdb593-4731-4c62-af40-389d7709f1eb-2,2020-04-15 15:12:45.202000+00:00,21deb68d-084a-4389-be5c-6cd13b9e3e9c,None,Very disappointed. Screen barely responds to t...,Very_Bad,"[0.12313512969153419, 0.11768187924517681, 0.1...",None
